In [10]:
from transformers import *
import torch

In [53]:
model_BBPC = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
tokenizer_BBPC = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")

model_HBC = AutoModelWithLMHead.from_pretrained("allegro/herbert-base-cased")
tokenizer_HBC = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

model_HLC = AutoModelWithLMHead.from_pretrained("allegro/herbert-large-cased")
tokenizer_HLC = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
models_with_tokenizers = {"dkleczek/bert-base-polish-cased-v1": (model_BBPC, tokenizer_BBPC),
                          "allegro/herbert-base-cased": (model_HBC, tokenizer_HBC),
                          "allegro/herbert-large-cased": (model_HLC, tokenizer_HLC)}

In [77]:
def display_top_predictions(sentences, top_num, mask_end=True):
    for name, model_tokenizer in models_with_tokenizers.items():
        model = model_tokenizer[0]
        tokenizer = model_tokenizer[1]

        print("Model name: {}".format(name))
        for sentence in sentences:
            if mask_end:
                sentence = sentence + "{}".format(tokenizer.mask_token) + "."
            else:
                sentence = sentence.replace("[MASK]", tokenizer.mask_token)

            print("\t" + sentence)

            input = tokenizer.encode(sentence, return_tensors="pt")
            mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

            token_logits = model(input, return_dict=True).logits
            mask_token_logits = token_logits[0, mask_token_index, :]
            top_5_tokens = torch.topk(mask_token_logits, top_num, dim=1).indices[0].tolist()
            for i, token in enumerate(top_5_tokens):
                print("\t\t{} : {}".format(i+1, tokenizer.decode([token])))
                i+=1

In [69]:
#zad 3 
sentences = ["Warszawa to największe ", "Te zabawki należą do ", "Policjant przygląda się ",
             "Na środku skrzyżowania widać ", "Właściciel samochodu widział złodzieja z ", 
             "Prezydent z premierem rozmawiali wczoraj o ", "Witaj drogi "]

display_top_predictions(sentences, 5)

Model name: dkleczek/bert-base-polish-cased-v1
	Warszawa to największe [MASK].
		1 : miasto
		2 : województwo
		3 : lotnisko
		4 : miasteczko
		5 : państwo
	Te zabawki należą do [MASK].
		1 : ciebie
		2 : mnie
		3 : nas
		4 : pana
		5 : niego
	Policjant przygląda się [MASK].
		1 : temu
		2 : sprawie
		3 : im
		4 : wszystkiemu
		5 : panu
	Na środku skrzyżowania widać [MASK].
		1 : rzekę
		2 : ulicę
		3 : drzewa
		4 : drogę
		5 : las
	Właściciel samochodu widział złodzieja z [MASK].
		1 : bronią
		2 : tyłu
		3 : ulicy
		4 : bliska
		5 : zewnątrz
	Prezydent z premierem rozmawiali wczoraj o [MASK].
		1 : tym
		2 : Polsce
		3 : budżecie
		4 : ASF
		5 : ustawie
	Witaj drogi [MASK].
		1 : chłopcze
		2 : przyjacielu
		3 : bracie
		4 : kolego
		5 : synu
Model name: allegro/herbert-base-cased
	Warszawa to największe <mask>.
		1 : miasto
		2 : lotnisko
		3 : centrum
		4 : miasta
		5 : atrakcje
	Te zabawki należą do <mask>.
		1 : rodziny
		2 : nas
		3 : nich
		4 : najlepszych
		5 : .
	Policjant pr

In [70]:
#zad 4
sentences = ["Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ",
             "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie  "]
display_top_predictions(sentences, 5)

Model name: dkleczek/bert-base-polish-cased-v1
	Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].
		1 : zgodził
		2 : dowiedział
		3 : martwił
		4 : bał
		5 : zabił
	Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie  [MASK].
		1 : zgodziła
		2 : bała
		3 : dowiedziała
		4 : zabiła
		5 : pojawiła
Model name: allegro/herbert-base-cased
	Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie <mask>.
		1 : zdziwił
		2 : poddał
		3 : dowiedział
		4 : zastanawiał
		5 : przyznał
	Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie  <mask>.
		1 : dowiedziała
		2 : przyznała
		3 : bała
		4 : śmiała
		5 : zmieniła
Model name: allegro/herbert-large-cased
	Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie <mask>.
		1 : bał
		2 : poddał
		3 : zabił
		4 : śmiał
		5 : zastanawiał
	Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie  <mask>.
		1 : bała
		2 : zgodziła
		3 : dowiedziała
		4 : zmieniła
		5 : śmiał

In [78]:
#zad 5
sentences = ["[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.",
             "W wakacje odwiedziłem [MASK], który jest stolicą Islandii.",
             "Informatyka na [MASK] należy do najlepszych kierunków w Polsce."]

display_top_predictions(sentences, 5, False)

Model name: dkleczek/bert-base-polish-cased-v1
	[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
		1 : Woda
		2 : Mięso
		3 : Słońce
		4 : Nie
		5 : Piwo
	W wakacje odwiedziłem [MASK], który jest stolicą Islandii.
		1 : kraj
		2 : Cypr
		3 : Meksyk
		4 : Gibraltar
		5 : Wellington
	Informatyka na [MASK] należy do najlepszych kierunków w Polsce.
		1 : wsi
		2 : świecie
		3 : żywo
		4 : pewno
		5 : odległość
Model name: allegro/herbert-base-cased
	<mask> wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
		1 : Woda
		2 : Słońce
		3 : Ziemia
		4 : Następnie
		5 : Ciało
	W wakacje odwiedziłem <mask>, który jest stolicą Islandii.
		1 : Kraków
		2 : Oslo
		3 : Londyn
		4 : Gdańsk
		5 : Toruń
	Informatyka na <mask> należy do najlepszych kierunków w Polsce.
		1 : pewno
		2 : AGH
		3 : UW
		4 : studiach
		5 : UMK
Model name: allegro/herbert-large-cased
	<mask> wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
	

***Which of the models produced the best results?***
* Każdy z modeli radził sobie lepiej lub gorzej w zależności od konretnego zdania. 
* Np. w zdaniu "Informatyka na <mask> należy do najlepszych kierunków w Polsce." modele allegro/herbert-large-cased, allegro/herbert-base-cased znalazły nazwy polskich uczelni w przeciwieńśtwie do dkleczek/bert-base-polish-cased-v1.
* W zdaniu "Witaj drogi <mask>." model HBC jako jedyny nie poradził sobie z wołaczem i zwrócił nieodmienione polskie imiona. 
* Model Bert i HLC poradziły sobie dobrze ze zdaniem "Właściciel samochodu widział złodzieja z [MASK]", gdzie przewidywały takie słowa jak bronią, nożem. 
* Trudno, więc podać jedną nazwę modelu, chociaż model dkleczek/bert-base-polish-cased-v1 miał tę przewagę, że nigdy nie przewidział '.' w przeciwieństwie do pozostałych modeli.
    
***Was any of the models able to capture Polish grammar?***
* W zadaniu 4 wszystkie modele poradziły sobie z rozróżnieniem rodzaju żeńskiego do męskiego. 
* Natomiast analizując zdanie "Warszawa to największe [MASK].", tylko model Bert zwrócił wszystkie słowa w liczbie pojedynczej.
* Model allegro/herbert-base-cased nie poradził sobie z rozpoznaniem wołacza. 
* Wydaje się, że model dkleczek/bert-base-polish-cased-v1 najtrafniej poradził sobie z rozpoznaniem polskiej gramatyki.
    
***Was any of the models able to capture long-distant relationships between the words?***
* Modelom allegro/herbert-large-cased, allegro/herbert-base-cased udało się powiązać nazwy polskich uczelni ze słowem informatyka oraz m.in. oddalonym słowem 'Polsce'.
* W zdaniu "W wakacje odwiedziłem <mask>, który jest stolicą Islandii.", wszystkie modele pozwiązały jakąś nazwę miasta ze "stolicą Islandii" (chociaż niekoniecznie trafnie). 
*W zdaniu "Prezydent z premierem rozmawiali wczoraj o <mask>." pierwsze dwa modele (chociaż drugi trochę lepiej) dobrze poradziły sobie z powiązaniem ostatniego słowa z tematyką policztyczną (za którą odpowiadają pierwsze słowa w zdaniu).
    
***Was any of the models able to capture world knowledge?***
* Tak, np. w zdaniu "[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza." wszystkie modele rozpoznały, że chodzi o wodę. 
* Chociaż żaden z nich nie odgadł prawdiłowo stolicy Islandii. 
* Natomiast w zdaniu "Informatyka na [MASK] należy do najlepszych kierunków w Polsce." dwa ostatnie modele uchwyciły wiedzę dziedzinową.
   
***What are the most striking errors made by the models?***
* Predykcja kropki, mimo tego, że widocznie oczekujemy kolejnego słowa.
* Predykcje pierwszego modelu dla zdania "Informatyka na [MASK] należy do najlepszych kierunków w Polsce." (zwrócenie wsi, żywo).